In [1]:
import duckdb
import time
import pandas as pd
from tqdm.auto import tqdm
import numpy as np

/home/newuser/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
con = duckdb.connect(database="tpch_sf1.db")

In [3]:
#6D version
query_template = """select
	sum(l_extendedprice* (1 - l_discount)) as revenue
from
	lineitem,
	part
where
	(
		p_partkey = l_partkey
		and p_brand = 'Brand#24'
		and p_container in ('SM CASE', 'SM BOX', 'SM PACK', 'SM PKG')
		and l_quantity >= '$quantity1a' and l_quantity <= '$quantity1b'
		and p_size between 1 and 5
		and l_shipmode in ('AIR', 'AIR REG')
		and l_shipinstruct = 'DELIVER IN PERSON'
	)
	or
	(
		p_partkey = l_partkey
		and p_brand = 'Brand#32'
		and p_container in ('MED BAG', 'MED BOX', 'MED PKG', 'MED PACK')
		and l_quantity >= '$quantity2a' and l_quantity <= '$quantity2b'
		and p_size between 1 and 10
		and l_shipmode in ('AIR', 'AIR REG')
		and l_shipinstruct = 'DELIVER IN PERSON'
	)
	or
	(
		p_partkey = l_partkey
		and p_brand = 'Brand#31'
		and p_container in ('LG CASE', 'LG BOX', 'LG PACK', 'LG PKG')
		and l_quantity >= '$quantity3a' and l_quantity <= '$quantity3b'
		and p_size between 1 and 15
		and l_shipmode in ('AIR', 'AIR REG')
		and l_shipinstruct = 'DELIVER IN PERSON'
	);"""


In [4]:
#3D version
query_template = """select
	sum(l_extendedprice* (1 - l_discount)) as revenue
from
	lineitem,
	part
where
	(
		p_partkey = l_partkey
		and p_brand = 'Brand#24'
		and p_container in ('SM CASE', 'SM BOX', 'SM PACK', 'SM PKG')
		and l_quantity >= '$quantity1'
		and p_size between 1 and 5
		and l_shipmode in ('AIR', 'AIR REG')
		and l_shipinstruct = 'DELIVER IN PERSON'
	)
	or
	(
		p_partkey = l_partkey
		and p_brand = 'Brand#32'
		and p_container in ('MED BAG', 'MED BOX', 'MED PKG', 'MED PACK')
		and l_quantity >= '$quantity2' 
		and p_size between 1 and 10
		and l_shipmode in ('AIR', 'AIR REG')
		and l_shipinstruct = 'DELIVER IN PERSON'
	)
	or
	(
		p_partkey = l_partkey
		and p_brand = 'Brand#31'
		and p_container in ('LG CASE', 'LG BOX', 'LG PACK', 'LG PKG')
		and l_quantity >= '$quantity3'
		and p_size between 1 and 15
		and l_shipmode in ('AIR', 'AIR REG')
		and l_shipinstruct = 'DELIVER IN PERSON'
	);"""

In [5]:
min_quantity = float(con.sql("SELECT MIN(l_quantity) FROM lineitem").fetchone()[0])
max_quantity = float(con.sql("SELECT MAX(l_quantity) FROM lineitem").fetchone()[0])
(min_quantity, max_quantity)

(1.0, 50.0)

In [6]:
table = []

# Loop over the date range for `quantity`
for quantity1 in tqdm(list(np.arange(min_quantity, max_quantity + 1.00, 1.00))):
    # Loop over the range for `quantity`
    for quantity2 in tqdm(list(np.arange(min_quantity, max_quantity + 1.00, 1.00)), leave=False):
        # Loop over the range for `quantity`
        for quantity3 in tqdm(list(np.arange(min_quantity, max_quantity + 1.00, 1.00)), leave=False):
            params = {'quantity1': quantity1, 'quantity2': quantity2, 'quantity3': quantity3}
            for key, val in params.items():
                query_template = query_template.replace(f"${key}", str(val))
            start = time.time()
            res = con.sql(query_template)
            elapsed = time.time() - start
            params['elapsed'] = elapsed
            table.append(params)

table = pd.DataFrame(table)
table

  0%|          | 0/50 [00:00<?, ?it/s]



































































































  2%|▏         | 1/50 [00:00<00:34,  1.43it/s]



































































































  4%|▍         | 2/50 [00:01<00:32,  1.46it/s]



































































































  6%|▌         | 3/50 [00:02<00:31,  1.48it/s]



































































































  8%|▊         | 4/50 [00:02<00:31,  1.45it/s]



































































































 10%|█         | 5/50 [00:03<00:31,  1.45it/s]



































































































 12%|█▏        | 6/50 [00:04<00:30,  1.45it/s]























































































,quantity1,quantity2,quantity3,elapsed
0,1.0,1.0,1.0,0.000492
1,1.0,1.0,2.0,0.000257
2,1.0,1.0,3.0,0.000212
3,1.0,1.0,4.0,0.000201
4,1.0,1.0,5.0,0.000198
...,...,...,...,...
124995,50.0,50.0,46.0,0.000235
124996,50.0,50.0,47.0,0.000235
124997,50.0,50.0,48.0,0.000233
124998,50.0,50.0,49.0,0.000285


In [ ]:
con.close()